In [ ]:
import os
import pickle
os.getcwd()
os.chdir('C:\\ML_PROJECTS')





In [ ]:

filename='vgg19_melanoma_10396.jpg_reduced_features_test.pkl'

with open(filename,'rb') as f:
    data=pickle.load(f)


print(data.shape)


In [ ]:
import pickle
import numpy as np
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier

base_models = [
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(kernel='linear', probability=True, random_state=42)),
    ('xgboost', XGBClassifier(eval_metric='logloss', random_state=42)),
    ('decision_tree', DecisionTreeClassifier(random_state=42))
]


final_model = LogisticRegression(random_state=42)

stacked_model = StackingClassifier(estimators=base_models, final_estimator=final_model)

l=[]
def get_feature_vector(image_name):
    x=0
    i=0
    f1=f'vgg16_{image_name}_reduced_features.pkl'
    with open(f1,'rb') as f:
        data1=pickle.load(f)
    f2=f'vgg19_{image_name}_reduced_features.pkl'
    with open(f2,'rb') as f:
        data2=pickle.load(f)
    f3=f'resnet50_{image_name}_reduced_features.pkl'
    with open(f3,'rb') as f:
        data3=pickle.load(f)
    f4=f'inceptionv3_{image_name}_reduced_features.pkl'
    with open(f4,'rb') as f:
        data4=pickle.load(f)
    f5=f'densenet_features_concatenated.pkl'
    with open(f5,'rb') as f:
        data5=pickle.load(f)
    f_v=get_f_v_densenet(data5,image_name)
    for j in range(1024):
        x=(data1[i][j]+data2[i][j]+data3[i][j]+data4[i][j]+f_v[j])/5
        l.append(x)
    return l    
   
        
        
def get_f_v_densenet(data5,image_name):
    for img,f_v in data5:
        if image_name in img:
           return f_v       

base_dir='C:\\ML_PROJECTS'

label_0_dir=os.path.join(base_dir,'label_0')
label_1_dir=os.path.join(base_dir,'label_1')

def search_label_0_dir(filename):
    for files in os.listdir(label_0_dir):
       if filename in files:
           return 1
    return 0

def search_label_1_dir(filename):
    for files in os.listdir(label_1_dir):
       if filename in files:
           return 1
    return 0

df=pd.read_excel('melanoma_org_1.xlsx')
label=[]
for idx,row in df.iterrows():
    image_name=row['image_name']
    print(image_name)
    filename=f'concatenated_features_{image_name}.pkl'
    f_v=get_feature_vector(image_name)
    arr=np.array(f_v)
    print('Feature vector avg--->:')
    print(arr)
    print(f'Size:{arr.shape}')
    x=search_label_0_dir(filename)
    y=search_label_1_dir(filename)
    print('label:\t')
    if x:
        print(0)
        label.append(0)
    elif y:
        print(1)
        label.append(1)

arr=np.array(f_v)

print(arr.shape)

reshaped_array=arr.reshape(2000,1024)

stacked_model.fit(reshaped_array,label)




    


In [ ]:
import os
os.chdir('C:\\ML_PROJECTS')



In [ ]:
import random
import cv2
path_benign_test='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\benign'
path_malignant_test='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\malignant'


def load_images_with_labels(path,label,num_samples):
    images=os.listdir(path)
    if num_samples>len(images):
        num_samples=len(images)
    selected_images=random.sample(images,num_samples)    
    labeled_images=[]
    for img in selected_images:
        img_path=os.path.join(path,img)
        img_arr=cv2.imread(img_path)
        if img_arr is not None:
            labeled_images.append((img,label))
    return labeled_images       

benign_images=load_images_with_labels(path_benign_test,0,1000)
malignant_images=load_images_with_labels(path_malignant_test,1,1000)

data_images_mod=benign_images+malignant_images

print(len(data_images_mod))

In [ ]:
import random

random.shuffle(data_images_mod)

for img,label in data_images_mod:
    print(label,img)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org.xlsx')


base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
   
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
 
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status==0:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\benign', img_name)
        else:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features


batch_size = 200
all_features = []


for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
 
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'vgg16_features_batch_test_mod_{i//batch_size}.pkl')
    
    gc.collect()


In [ ]:
import pickle
import numpy as np
data_list=[]

for i in range(4):
    filename=f"vgg16_features_batch_test_mod_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)

concatenated_data = np.concatenate(data_list, axis=0)

with open('vgg16_features_concatenated_test.pkl', 'wb') as f:
    pickle.dump(concatenated_data, f)

print("Concatenation complete and saved to 'vgg16_features_concatenated_test.pkl'")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org.xlsx')


base_model = VGG19(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
   
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
   
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status==0:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\benign', img_name)
        else:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features


batch_size = 200
all_features = []


for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
 
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'vgg19_features_batch_test_{i//batch_size}.pkl')
   
    gc.collect()




In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(4):
    filename=f"vgg19_features_batch_test_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('vgg19_features_concatenated_test.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'vgg19_features_concatenated_test.pkl'")


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org.xlsx')


base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
    
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
   
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status == 0:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\benign', img_name)
        else:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features


batch_size = 200
all_features = []


for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
   
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'densenet_features_batch_test_{i//batch_size}.pkl')
    
    gc.collect()

print("Feature extraction complete and saved.")


In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(4):
    filename=f"densenet_features_batch_test_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('densenet_features_concatenated_test.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'densenet_features_concatenated_test.pkl'")


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org.xlsx')


base_model = InceptionV3(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status == 0:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\benign', img_name)
        else:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features

batch_size = 200
all_features = []

for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
    
   
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'inceptionv3_features_batch_test_{i//batch_size}.pkl')
    
   
    gc.collect()


In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(4):
    filename=f"inceptionv3_features_batch_test_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('inceptionv3_features_concatenated_test.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'inceptionv3_features_concatenated_test.pkl'")


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org.xlsx')


base_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status == 0:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\benign', img_name)
        else:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\test\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features

batch_size = 200
all_features = []

for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
    
    
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'resnet50_features_batch_test_{i//batch_size}.pkl')
    
    
    gc.collect()


In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(4):
    filename=f"resnet50_features_batch_test_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('resnet50_features_concatenated_test.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'resnet50_features_concatenated_test.pkl'")


In [ ]:
import pickle
import numpy as np

with open('resnet50_features_concatenated_test.pkl','rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'image:{images},{f_v.shape}')

In [ ]:
import pickle
import numpy as np

with open('densenet_features_concatenated.pkl','rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'image:{images},{f_v.shape}')

In [ ]:
import pickle
import numpy as np

with open('vgg16_features_concatenated.pkl','rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'image:{images},{f_v.shape}')

In [ ]:
import pickle
import numpy as np

with open('vgg19_features_concatenated.pkl','rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'image:{images},{f_v.shape}')

In [ ]:
import pickle
import numpy as np

with open('inceptionv3_features_concatenated.pkl','rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'image:{images},{f_v.shape}')

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(210,210,3)))  
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(1024, activation='relu'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='inceptionv3_features_concatenated_test.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    # print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,1228),mode='constant')
    num_samples=1
    height=210
    width=210
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model2.predict(reshaped_array)
    filename=f'inceptionv3_{image}_reduced_features_test.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model3 = models.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(182,183,3)))  
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Flatten())
model3.add(layers.Dense(1024, activation='relu'))
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='resnet50_features_concatenated_test.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    #print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,115),mode='constant')
    num_samples=1
    height=183
    width=183
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model3.predict(reshaped_array)
    filename=f'resnet50_{image}_reduced_features_test.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(92,92,3)))  
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Flatten())
model1.add(layers.Dense(1024, activation='relu'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='vgg16_features_concatenated_test.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    # print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,304),mode='constant')
    num_samples=1
    height=92
    width=92
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model1.predict(reshaped_array)
    filename=f'vgg16_{image}_reduced_features_test.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(92,92,3)))  
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Flatten())
model1.add(layers.Dense(1024, activation='relu'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='vgg19_features_concatenated_test.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    # print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,304),mode='constant')
    num_samples=1
    height=92
    width=92
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model1.predict(reshaped_array)
    filename=f'vgg19_{image}_reduced_features_test.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import os
os.chdir("C:\\ML_PROJECTS")

In [ ]:
import pickle
import pandas as pd
import os

df=pd.read_excel("melanoma_org.xlsx")
l_f_test=[]
def get_feature(image_name):
    i=0
    x=0
    f_vgg16_test=f"vgg16_{image_name}_reduced_features_test.pkl"
    f_vgg19_test=f"vgg19_{image_name}_reduced_features_test.pkl"
    f_inceptionv3_test=f"inceptionv3_{image_name}_reduced_features_test.pkl"
    f_resnet50_test=f"resnet50_{image_name}_reduced_features_test.pkl"
    filename="densenet_features_concatenated_test.pkl"
    with open(f_vgg16_test,'rb') as f:
        data_vgg16=pickle.load(f)
    with open(f_vgg19_test,'rb') as f:
        data_vgg19=pickle.load(f)
    with open(f_inceptionv3_test,'rb') as f:
        data_inceptionv3=pickle.load(f)
    with open(f_resnet50_test,'rb') as f:
        data_resnet50=pickle.load(f)
    with open(filename,'rb') as f:
        data_densenet=pickle.load(f)
    f_v=get_densenet_test(data_densenet,image_name)
    
    for j in range(1024):
        x=(data_vgg16[i][j]+data_vgg19[i][j]+data_inceptionv3[i][j]+data_resnet50[i][j]+f_v[j])/5
        l_f_test.append(x)
    print(f"Feature_vector_merged for {image_name}:{x}")
    
        

def get_densenet_test(data_densenet,image_name):
    for img,f_v in data_densenet:
        if image_name in img:
            return f_v
        
            
l_test=[]
count=0
for idx,row in df.iterrows():
    image_name=row['image_name']
    label=row['status']
    get_feature(image_name)
    l_test.append(label)
    print(label)
    count+=1
    if count==800:
        break

test_array=np.array(l_f_test)
mod_test_array=test_array.reshape(800,1024)
print('array reshaped succesfully')
print(mod_test_array)
print(l_test)



In [ ]:
import pickle
import numpy as np
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier

base_models = [
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(kernel='linear', probability=True, random_state=42)),
    ('xgboost', XGBClassifier(eval_metric='logloss', random_state=42)),
    ('decision_tree', DecisionTreeClassifier(random_state=42))
]


final_model = LogisticRegression(random_state=42)

stacked_model = StackingClassifier(estimators=base_models, final_estimator=final_model)

l=[]
def get_feature_vector(image_name):
    x=0
    i=0
    f1=f'vgg16_{image_name}_reduced_features.pkl'
    with open(f1,'rb') as f:
        data1=pickle.load(f)
    f2=f'vgg19_{image_name}_reduced_features.pkl'
    with open(f2,'rb') as f:
        data2=pickle.load(f)
    f3=f'resnet50_{image_name}_reduced_features.pkl'
    with open(f3,'rb') as f:
        data3=pickle.load(f)
    f4=f'inceptionv3_{image_name}_reduced_features.pkl'
    with open(f4,'rb') as f:
        data4=pickle.load(f)
    f5=f'densenet_features_concatenated.pkl'
    with open(f5,'rb') as f:
        data5=pickle.load(f)
    f_v=get_f_v_densenet(data5,image_name)
    for j in range(1024):
        x=(data1[i][j]+data2[i][j]+data3[i][j]+data4[i][j]+f_v[j])/5
        l.append(x)
    return l    
   
        
        
def get_f_v_densenet(data5,image_name):
    for img,f_v in data5:
        if image_name in img:
           return f_v       

base_dir='C:\\ML_PROJECTS'

label_0_dir=os.path.join(base_dir,'label_0')
label_1_dir=os.path.join(base_dir,'label_1')

def search_label_0_dir(filename):
    for files in os.listdir(label_0_dir):
       if filename in files:
           return 1
    return 0

def search_label_1_dir(filename):
    for files in os.listdir(label_1_dir):
       if filename in files:
           return 1
    return 0

df=pd.read_excel('melanoma_org_1.xlsx')
label=[]
for idx,row in df.iterrows():
    image_name=row['image_name']
    print(image_name)
    filename=f'concatenated_features_{image_name}.pkl'
    f_v=get_feature_vector(image_name)
    arr=np.array(f_v)
    print('Feature vector avg--->:')
    print(arr)
    print(f'Size:{arr.shape}')
    x=search_label_0_dir(filename)
    y=search_label_1_dir(filename)
    print('label:\t')
    if x:
        print(0)
        label.append(0)
    elif y:
        print(1)
        label.append(1)

arr=np.array(f_v)

print(arr.shape)

reshaped_array=arr.reshape(2000,1024)

stacked_model.fit(reshaped_array,label)

    


In [ ]:
y_pred=stacked_model.predict(mod_test_array)
accuracy = accuracy_score(l_test, y_pred)
print(f'Accuracy of the stacked model: {accuracy:.2f}')